In [79]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats

brutos = pd.read_csv('dados.csv')
brutos.head(10)

,Year,Leading Cause,Sex,Race Ethnicity,Deaths,Death Rate,Age Adjusted Death Rate
0,2010,Influenza (Flu) and Pneumonia (J09-J18),F,Hispanic,228,18.7,23.1
1,2008,"Accidents Except Drug Posioning (V01-X39, X43,...",F,Hispanic,68,5.8,6.6
2,2013,"Accidents Except Drug Posioning (V01-X39, X43,...",M,White Non-Hispanic,271,20.1,17.9
3,2010,Cerebrovascular Disease (Stroke: I60-I69),M,Hispanic,140,12.3,21.4
4,2009,"Assault (Homicide: Y87.1, X85-Y09)",M,Black Non-Hispanic,255,30,30
5,2012,Mental and Behavioral Disorders due to Acciden...,F,Other Race/ Ethnicity,.,.,.
6,2012,Cerebrovascular Disease (Stroke: I60-I69),F,Asian and Pacific Islander,102,17.5,20.7
7,2009,Essential Hypertension and Renal Diseases (I10...,M,Asian and Pacific Islander,26,5.1,7.2
8,2010,All Other Causes,F,White Non-Hispanic,2140,149.7,93.9
9,2009,Alzheimer's Disease (G30),F,Other Race/ Ethnicity,.,.,.


In [111]:
nomes={
'Malignant Neoplasms (Cancer: C00-C97)':'MN',
'Diseases of Heart (I00-I09, I11, I13, I20-I51)':'DH',
'All Other Causes':'AOC',
'Influenza (Flu) and Pneumonia (J09-J18)':'IP',
'Diabetes Mellitus (E10-E14)':'D',
'Cerebrovascular Disease (Stroke: I60-I69)':'CD',
'Chronic Lower Respiratory Diseases (J40-J47)':'LR',
'Accidents Except Drug Posioning (V01-X39, X43, X45-X59, Y85-Y86)':'AEDP',
'Essential Hypertension and Renal Diseases (I10, I12)':'HR',
'Human Immunodeficiency Virus Disease (HIV: B20-B24)':'HIV',
'Mental and Behavioral Disorders due to Accidental Poisoning and Other Psychoactive Substance Use (F11-F16, F18-F19, X40-X42, X44)':'MBD',
'Intentional Self-Harm (Suicide: X60-X84, Y87.0)':'IH',
"Alzheimer's Disease (G30)":'AD',
'Chronic Liver Disease and Cirrhosis (K70, K73)':'CLD',
'Certain Conditions originating in the Perinatal Period (P00-P96)':'PP',
'Assault (Homicide: Y87.1, X85-Y09)':'As',
'Nephritis, Nephrotic Syndrome and Nephrisis (N00-N07, N17-N19, N25-N27)':'NNN',
'Septicemia (A40-A41)':'S',
'Congenital Malformations, Deformations, and Chromosomal Abnormalities (Q00-Q99)':'CA',
'Viral Hepatitis (B15-B19)':'VH',
'Insitu or Benign / Uncertain Neoplasms (D00-D48)':'BN',
'Aortic Aneurysm and Dissection (I71)':'AAD',
'Atherosclerosis (I70)':'At',
'Mental and Behavioral Disorders due to Use of Alcohol (F10)':'BA',
'Tuberculosis (A16-A19)':'T',
"Parkinson's Disease (G20)":'P'
}
def soma_mortes(causa,sexo,raca,d):
    if d!='.':
        causa=nomes[causa]
        chave=causa+'|'+sexo+'|'+raca
        if chave not in mortes:
            mortes[chave]=int(d)
        else:
            mortes[chave]=mortes[chave]+int(d)
def csv(dic):
    tudo='Causa, Sexo, Etnia, Mortes\n'
    
    
    for chave,d in dic.items():
        e=chave.split('|')
        for i in e:
            tudo+=i+','
        tudo+=str(d)+'\n'
    return tudo

In [112]:
mortes={}
linhas=[]
for i in range(len(brutos)):
    linhas.append(list(brutos.iloc[i, :]))
data=[]
for l in linhas:
    soma_mortes(l[1],l[2],l[3],l[4])

arquivo=csv(mortes)


In [113]:
print (arquivo)

Causa, Sexo, Etnia, Mortes
IP,F,Hispanic,1710
AEDP,F,Hispanic,437
AEDP,M,White Non-Hispanic,2206
CD,M,Hispanic,1132
As,M,Black Non-Hispanic,1968
CD,F,Asian and Pacific Islander,677
HR,M,Asian and Pacific Islander,221
AOC,F,White Non-Hispanic,17272
CD,M,Black Non-Hispanic,1502
CD,F,Black Non-Hispanic,2147
DH,F,Not Stated/Unknown,691
IH,M,Not Stated/Unknown,30
HIV,F,Hispanic,467
CLD,M,Other Race/ Ethnicity,15
DH,M,Black Non-Hispanic,15887
AD,F,Hispanic,605
IP,M,Hispanic,1640
AEDP,M,Not Stated/Unknown,58
HR,M,White Non-Hispanic,1215
DH,M,Asian and Pacific Islander,4080
HR,F,Hispanic,764
AOC,F,Other Race/ Ethnicity,230
MN,F,Other Race/ Ethnicity,267
AOC,F,Black Non-Hispanic,11178
CLD,M,Hispanic,1148
CD,F,White Non-Hispanic,3250
HIV,M,Black Non-Hispanic,2259
MN,M,White Non-Hispanic,26041
MN,M,Black Non-Hispanic,12361
LR,F,Not Stated/Unknown,53
DH,M,White Non-Hispanic,37249
D,M,White Non-Hispanic,2125
IP,F,Black Non-Hispanic,2288
PP,F,Not Stated/Unknown,55
HR,F,Other Race/ Ethnicity,8
HR,F,A

In [115]:
with open ('limpo.csv','w') as f:
    f.write(arquivo)